## Overview
## Outline
1. Import Library
2. Setup pre-requisites
3. Extract dataset to images
4. Upload images to Azure data store
4. Setting up Azure ML Infrastructure

### 1. Import library

In [11]:
## Install needed packages
%pip install opencv-python torch torchvision torchaudio azureml-core

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [33]:
import os
from importlib import reload

import cv2
from azureml.core import Workspace, Dataset, Datastore

## Using torchvision to create a dataset
from torchvision.datasets import ImageFolder
from torchvision import transforms
import torchvision
import torch


### 2. Setup pre-requisites

Clone source code from github

In [13]:
## curl source code from github
!wget 'https://github.com/nnvu-fit/iusai-project/archive/refs/heads/main.zip' -O main.zip
!unzip main.zip
!mv iusai-project-main/* .
!rm -rf iusai-project-main main.zip

'wget' is not recognized as an internal or external command,
operable program or batch file.
'unzip' is not recognized as an internal or external command,
operable program or batch file.
'mv' is not recognized as an internal or external command,
operable program or batch file.


'rm' is not recognized as an internal or external command,
operable program or batch file.


In [34]:
## Setup Azure ML Workspace
ws = Workspace.from_config()
## From workspace, get/create the default datastore
ds = ws.get_default_datastore()
ws, ds

(Workspace.create(name='ws-vunn-iusai-sea-sp6k7', subscription_id='d554f489-6933-4c33-8722-a536b3682bd7', resource_group='rg-vunn-iusai-sea-sp6k7'),
 {
   "name": "workspaceblobstore",
   "container_name": "azureml-blobstore-5fddeec9-0fbc-4996-b95f-26996d3f3bcd",
   "account_name": "wsvunniusaisea7140421596",
   "protocol": "https",
   "endpoint": "core.windows.net"
 })

### 3. Extract dataset to images

In [3]:
## define videos location + images output location
video_path = 'videos/'
images_path = 'images/'
print('video_path: ', video_path)
print('images_path: ', images_path)

video_path:  videos/
images_path:  images/


In [4]:
import v2i
reload(v2i)
from v2i import extract_images_from_videos
import threading

def extract_images_from_videos_collection(video_path, images_path):
    ## get all videos file in video_path
    video_list_location_collection = os.listdir(video_path)

    ## define total_label_dict
    ## create threading pool
    thread = []
    ## for each video file
    for video_list_location in video_list_location_collection:
        ## check if video_location is not a directory (i.e. is a file), then skip
        if not os.path.isdir(video_path + video_list_location):
            continue
        ## list videos in video_location
        thread.append(threading.Thread(target=extract_images_from_videos, args=(video_path + video_list_location, images_path, 1)))
        # extract_images_from_videos(video_path + video_list_location, images_path, inteval=1)

    ## start all threads
    for t in thread: t.start()
    ## wait for all threads to finish
    for t in thread:
        t.join()
    
    ## check images in images_path
    images_list_location_collection = os.listdir(images_path)
    print('images_list_location_collection: ', images_list_location_collection)

# video_subject_6_path = video_path + '/subject6/'
# ## list videos in video_location
# label_dict = extract_images_from_videos(video_subject_6_path, images_path)

## extract images from videos
extract_images_from_videos_collection(video_path, images_path)


Label dict of videos/subject10/huangpi21.avi (start): [['0', 0], ['2', 0], ['1', 0]]
Label dict of videos/subject1/Yousef1.avi (start): [['0', 0], ['1', 0], ['2', 0], ['5', 0], ['6', 0], ['3', 0]]
Label dict of videos/subject11/alhashe31.avi (start): [['0', 0], ['2', 0]]
Label dict of videos/subject13/guolingh1.avi (start): [['0', 0], ['6', 0], ['2', 0], ['1', 0]]
Label dict of videos/subject12/makiluke1.avi (start): [['0', 0], ['2', 0]]
Label dict of videos/subject14/chuangy61.avi (start): [['0', 0], ['2', 0], ['1', 0]]
Label dict of videos/subject15/bajajpak1.avi (start): [['0', 0], ['2', 0], ['1', 0]]
Label dict of videos/subject16/dhingra51.avi (start): [['0', 0], ['2', 0], ['1', 0]]
Label dict of videos/subject17/zhoulian1.avi (start): [['0', 0], ['1', 0], ['2', 0]]
Label dict of videos/subject18/yuqianyi1.avi (start): [['0', 0], ['2', 0], ['1', 0]]
Label dict of videos/subject19/wangyus11.avi (start): [['0', 0], ['2', 0], ['1', 0]]


Label dict of videos/subject21/Mark1.avi (start): [['0', 0], ['1', 0], ['2', 0]]
Label dict of videos/subject23/vahid1.avi (start): [['0', 0], ['1', 0], ['2', 0]]
Label dict of videos/subject22/yarub1.avi (start): [['0', 0], ['1', 0], ['2', 0]]
Label dict of videos/subject2/Jourabloo1.avi (start): [['0', 0], ['6', 0], ['1', 0], ['2', 0], ['3', 0]]
Label dict of videos/subject20/Cool1.avi (start): [['0', 0], ['2', 0], ['1', 0]]
Label dict of videos/subject24/mustaffa1.avi (start): [['0', 0], ['1', 0], ['5', 0], ['2', 0]]
Label dict of videos/subject4/meowseph1.avi (start): [['0', 0], ['1', 0], ['2', 0], ['3', 0]]
Label dict of videos/subject3/XiYin1.avi (start): [['0', 0], ['1', 0], ['2', 0], ['6', 0], ['3', 0]]
Label dict of videos/subject5/Muath1.avi (start): [['0', 0], ['1', 0], ['2', 0], ['6', 0], ['3', 0]]
Label dict of videos/subject6/husseinhijazi1.avi (start): [['0', 0], ['2', 0], ['1', 0], ['5', 0], ['6', 0], ['3', 0]]
Label dict of videos/subject7/zach1.avi (start): [['0', 0],

Exception in thread Thread-10 (extract_images_from_videos):
Traceback (most recent call last):
  File "c:\Users\nnvuf\AppData\Local\Programs\Python\Python310\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "c:\Users\nnvuf\AppData\Local\Programs\Python\Python310\lib\threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "c:\Users\nnvuf\source\ai-final-project\v2i.py", line 67, in extract_images_from_videos
    cv2.destroyAllWindows()
cv2.error: OpenCV(4.8.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:1266: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvDestroyAllWindows'



### 4. Upload images to Azure data store

In [ ]:
## upload images to data asset using Dataset.File.upload_directory
Dataset.File.upload_directory(src_dir=images_path, target=(ds, 'images-extra-small'), overwrite=True, show_progress=True) \
    .register(workspace=ws, name='images-extra-small', description='images-extra-small') ## register dataset \\


Validating arguments.
Arguments validated.
'overwrite' is set to True. Any file already present in the target will be overwritten.
Uploading files from 'c:/Users/nnvuf/source/ai-final-project/images' to 'images-extra-small'
Creating new dataset


{
  "source": [
    "('workspaceblobstore', '/images-extra-small')"
  ],
  "definition": [
    "GetDatastoreFiles"
  ],
  "registration": {
    "id": "22c174ee-e3e7-4e60-a949-e20eb237c73d",
    "name": "images-extra-small",
    "version": 1,
    "description": "images-extra-small",
    "workspace": "Workspace.create(name='ws-vunn-iusai-sea-sp6k7', subscription_id='d554f489-6933-4c33-8722-a536b3682bd7', resource_group='rg-vunn-iusai-sea-sp6k7')"
  }
}

### 5. Setup public workspace endpoint

In [ ]:
dataset_collection_list = ['images-extra-small', 'images-small-v2', 'images-medium']
## register dataset using Dataset.File.from_files
for dataset_collection in dataset_collection_list:
    ## check if dataset_collection is already registered
    if dataset_collection in ws.datasets.keys():
        print('dataset_collection: ', dataset_collection, ' is already registered')
        continue
    Dataset.File.from_files(path=(ds, dataset_collection)) \
        .register(workspace=ws, name=dataset_collection, description=dataset_collection) ## register dataset \\


dataset_collection:  images-extra-small  is already registered
dataset_collection:  images-small-v2  is already registered
dataset_collection:  images-medium  is already registered


## Define and Train models
1. ResNET
2. DenseNET

### Setup device + load dataset

In [35]:
## import train and scrore function from train.py
import train as t;
reload(t)
from train import train, score_model, get_device

device = get_device()
device

'cuda'

In [36]:
images_path = 'images'
images_ds_path = "images-extra-small"
if not os.path.exists(images_path):
    os.mkdir(images_path)
# download data asset to local if images_path is empty
if len(os.listdir(images_path)) == 0:
    print('images_path is empty, download images_ds to images_path')
    # download data asset to local
    images_ds = Dataset.get_by_name(workspace=ws, name=images_ds_path)
    images_ds.download(target_path=images_path, overwrite=True)

#### Prepare Dataset

In [37]:
from torch.utils.data import DataLoader
import data_set as ds
reload(ds)

transform = transforms.Compose([transforms.Resize((224,224)), transforms.ToTensor()])
imageDataset = ds.ImageDataset('images', transform=transform)
## define batch_size
batch_size = 64

In [38]:
image, label = imageDataset.get_image(0)
# image.show()
labels = imageDataset.labels()
## show labels in Interger
print('labels: ',  [int(l) for l in labels])

labels:  [0, 1, 2, 3, 5, 6]


In [39]:
## split dataset into train and test dataset using random_split
from torch.utils.data import random_split
train_val_size = int(0.9 * len(imageDataset))
test_size = len(imageDataset) - train_val_size
train_val_ds, test_ds = random_split(imageDataset, [train_val_size, test_size])

## split train_val_ds into train_ds and val_ds using random_split
train_size = int(0.8 * len(train_val_ds))
val_size = len(train_val_ds) - train_size
train_ds, val_ds = random_split(train_val_ds, [train_size, val_size])

train_dataloader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=0)
val_dataloader = DataLoader(val_ds, batch_size=batch_size, shuffle=True, num_workers=0)
test_dataloader = DataLoader(test_ds, batch_size=batch_size, shuffle=True, num_workers=0)

print('train_ds: ', len(train_ds))
print('val_ds: ', len(val_ds))
print('test_ds: ', len(test_ds))


train_ds:  18109
val_ds:  4528
test_ds:  2516


### 1. Resnes18

In [40]:
## get resnet model of image classification from torchvision
model = torchvision.models.resnet18(pretrained=True)
## define optimizer using Adam and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = torch.nn.CrossEntropyLoss()
optimizer, loss_fn

(Adam (
 Parameter Group 0
     amsgrad: False
     betas: (0.9, 0.999)
     capturable: False
     differentiable: False
     eps: 1e-08
     foreach: None
     fused: None
     lr: 0.001
     maximize: False
     weight_decay: 0
 ),
 CrossEntropyLoss())

In [41]:
## train model
train(model, optimizer, loss_fn, train_dataloader, val_dataloader, epochs=1, device=device)
## score model
score_model(model, loss_fn, test_dataloader, device=device)

KeyboardInterrupt: 

In [ ]:
## train model using device (CPU or GPU) 10 epoch
train(model, optimizer, loss_fn, train_dataloader, val_dataloader, epochs=100, device=device)
## score model
score_model(model, loss_fn, test_dataloader, device=device)

Epoch 1/10, Train Loss: 0.4070, Test Loss: 0.4468, Total Time: 00 hours, 10 minutes, 50 seconds
Epoch 2/10, Train Loss: 0.2953, Test Loss: 0.3624, Total Time: 00 hours, 12 minutes, 53 seconds
Epoch 3/10, Train Loss: 0.2196, Test Loss: 0.3772, Total Time: 00 hours, 13 minutes, 49 seconds
Epoch 4/10, Train Loss: 0.1600, Test Loss: 0.3560, Total Time: 00 hours, 16 minutes, 10 seconds
Epoch 5/10, Train Loss: 0.1331, Test Loss: 0.3690, Total Time: 00 hours, 15 minutes, 54 seconds
Epoch 6/10, Train Loss: 0.1037, Test Loss: 0.3158, Total Time: 00 hours, 15 minutes, 15 seconds
Epoch 7/10, Train Loss: 0.0790, Test Loss: 0.3453, Total Time: 00 hours, 15 minutes, 53 seconds
Epoch 8/10, Train Loss: 0.0761, Test Loss: 0.4047, Total Time: 00 hours, 15 minutes, 15 seconds
Epoch 9/10, Train Loss: 0.0618, Test Loss: 0.3804, Total Time: 00 hours, 15 minutes, 21 seconds
Epoch 10/10, Train Loss: 0.0561, Test Loss: 0.4284, Total Time: 00 hours, 16 minutes, 02 seconds


0.4667215104080109

In [ ]:
## save model
torch.save(model.state_dict(), 'model_resnes18.100_epochs.small.pth')

### 2. DenseNet

In [ ]:
## get denseNet model of image classification from torchvision
model = torchvision.models.densenet121(pretrained=True, num_classes=len(labels))
## define optimizer using Adam and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = torch.nn.CrossEntropyLoss()
optimizer, loss_fn

c:\Users\nnvuf\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


(Adam (
 Parameter Group 0
     amsgrad: False
     betas: (0.9, 0.999)
     capturable: False
     differentiable: False
     eps: 1e-08
     foreach: None
     fused: None
     lr: 0.001
     maximize: False
     weight_decay: 0
 ),
 CrossEntropyLoss())

In [21]:
## train model using device (CPU or GPU) 1 epoch
train(model, optimizer, loss_fn, train_dataloader, val_dataloader, epochs=1, device=device)
## score model
score_model(model, loss_fn, test_dataloader, device=device)

Epoch 1/1, Train Loss: 0.6834, Test Loss: 0.7046, Total Time: 05 hours, 27 minutes, 36 seconds


0.7259557617677225

In [22]:
## train model using device (CPU or GPU) 10 epoch
train(model, optimizer, loss_fn, train_dataloader, val_dataloader, epochs=10, device=device)
## score model
score_model(model, loss_fn, test_dataloader, device=device)

Epoch 1/10, Train Loss: 0.4411, Test Loss: 0.6275, Total Time: 05 hours, 34 minutes, 19 seconds
Epoch 2/10, Train Loss: 0.3439, Test Loss: 0.6895, Total Time: 05 hours, 34 minutes, 47 seconds
Epoch 3/10, Train Loss: 0.2888, Test Loss: 0.4157, Total Time: 05 hours, 43 minutes, 16 seconds
Epoch 4/10, Train Loss: 0.2428, Test Loss: 0.4163, Total Time: 05 hours, 35 minutes, 47 seconds
Epoch 5/10, Train Loss: 0.2049, Test Loss: 0.3158, Total Time: 07 hours, 43 minutes, 43 seconds
Epoch 6/10, Train Loss: 0.1741, Test Loss: 0.5003, Total Time: 08 hours, 44 minutes, 26 seconds
Epoch 7/10, Train Loss: 0.1587, Test Loss: 0.3826, Total Time: 08 hours, 34 minutes, 52 seconds
Epoch 8/10, Train Loss: 0.1292, Test Loss: 0.3596, Total Time: 08 hours, 28 minutes, 02 seconds
Epoch 9/10, Train Loss: 0.1110, Test Loss: 0.4189, Total Time: 17 hours, 53 minutes, 29 seconds
Epoch 10/10, Train Loss: 0.1113, Test Loss: 0.3861, Total Time: 08 hours, 27 minutes, 54 seconds


0.41211051089964534

In [23]:
## save model
torch.save(model.state_dict(), 'model_densenet121.small.pth')

### 3. Using Yolov5 to extract face then using ResNet to classification the output

In [ ]:
## use yoloresnet model
import model
reload(model)
from model import YoloResnet
model = YoloResnet()
## define optimizer using Adam and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = torch.nn.CrossEntropyLoss()
optimizer, loss_fn

In [ ]:
## train model using device (CPU or GPU) 1 epoch
train(model, optimizer, loss_fn, train_dataloader, val_dataloader, epochs=1, device=device)
## score model
score_model(model, loss_fn, test_dataloader, device=device)

In [ ]:
## train model using device (CPU or GPU) 10 epoch
train(model, optimizer, loss_fn, train_dataloader, val_dataloader, epochs=10, device=device)
## score model
score_model(model, loss_fn, test_dataloader, device=device)

In [ ]:
## save model
torch.save(model.state_dict(), 'model_yoloresnet.pth')